In [224]:
# Imports
import pandas as pd
import numpy as np
import requests # For the currency exchange rates

In [225]:
# First, we import the data
file = "salary_responses.csv"
data = pd.read_csv(file)
c = CurrencyRates()

In [226]:
# Data cleaning, validation
# We delete the columns that are not helpful for our purposes
data = data.drop( data.columns[[0,4,9]],axis=1)
# We now change the names of the columns to make manipulation easier
data.columns = ["Age", "Industry", "Job Title", "Annual Salary", "Bonus", 
                "Currency", "Alternate Currency", "Country", "US State", "US City", 
                "Work experience", "Work experience in field", "Education", "Gender", "Race"]

In [227]:
# Now we drop NA values from some columns, such as Annual Salary, Education, Race, and Gender, 
# as these columns are useful for predictions and model creation
data.dropna(subset=["Annual Salary", "Education", "Race", "Gender"], inplace=True)

In [228]:
# As we can see here: 
type(data["Annual Salary"][0])
# The type for Annual Salary is string, which is not ideal when we want to add these numbers
# So we need to change the type from string to numeric
data["Annual Salary"] = data["Annual Salary"].apply(lambda f: float(f.split()[0].replace(',', '')))
data["Annual Salary"] = data["Annual Salary"].astype(float)

# Now we need to change the NaN values from Bonus to 0
data["Bonus"] = data["Bonus"].fillna(0)
data["Total"] = data["Annual Salary"] + data["Bonus"]

# Now that the total has been calculated, we do not need the columns Bonus and Annual Salary
data.drop(["Annual Salary", "Bonus"], axis = 1, inplace=True)

In [231]:
# The other problem is the value "Other". These observations use a currency in the "Alternate Currency" column. But this column
# allowed for user input, which, as we know, is not ideal. 
# As we can see, the values are too varied, written not in currency code, and explanations instead of the currency
# First, we set everything to uppercase, to avoid issues with some written in lower and others in uppercase
data["Alternate Currency"] = data["Alternate Currency"].str.upper()
data["Alternate Currency"] = data["Alternate Currency"].str.strip()
data["Alternate Currency"] = data["Alternate Currency"].fillna("NONE")
data.loc[data["Alternate Currency"].str.contains("ARGENT"), "Alternate Currency"] = "ARS"
data.loc[data["Alternate Currency"].str.contains("MEXIC"), "Alternate Currency"] = "MXN"
data.loc[data["Alternate Currency"].str.contains("IND"), "Alternate Currency"] = "INR"
data.loc[data["Alternate Currency"].str.contains("ZWOTY"), "Alternate Currency"] = "PLN"
data.loc[data["Alternate Currency"].str.contains("POLISH"), "Alternate Currency"] = "PLN"
data.loc[data["Alternate Currency"].str.contains("AMERI"), "Alternate Currency"] = "USD"
data.loc[data["Alternate Currency"].str.contains("CANA"), "Alternate Currency"] = "CAD"
data.loc[data["Alternate Currency"].str.contains("CHIN"), "Alternate Currency"] = "CNY"
data.loc[data["Alternate Currency"].str.contains("NIS"), "Alternate Currency"] = "ILS"
data.loc[data["Alternate Currency"].str.contains("TAIW"), "Alternate Currency"] = "TWD"
data.loc[data["Alternate Currency"].str.contains("PHIL"), "Alternate Currency"] = "PHP"
data.loc[data["Alternate Currency"].str.contains("KOREAN"), "Alternate Currency"] = "KRW"
data.loc[data["Alternate Currency"].str.contains("ISRAEL"), "Alternate Currency"] = "ILS"
data.loc[data["Alternate Currency"].str.contains("AUSTRA"), "Alternate Currency"] = "AUD"
data.loc[data["Alternate Currency"].str.contains("THAI"), "Alternate Currency"] = "THB"
data.loc[data["Alternate Currency"].str.contains("CROAT"), "Alternate Currency"] = "HRK"
data.loc[data["Alternate Currency"].str.contains("SINGAP"), "Alternate Currency"] = "SGD"
data.replace({"Alternate Currency": {"BR$":"BRL","BRL (R$)" : "BRL","CZECH CROWNS" : "CZK", "NORWEGIAN KRONER (NOK)" : "NOK", 
                                "US DOLLAR" : "USD", "EURO" : "EUR", "NTD" : "TWD"}}, inplace=True)
# After replacing these values, we can delete everything that is over 3 characters in length, or less than 3, 
# as these are either not valid currencies, or not clear enough to use
data.drop(data[data["Alternate Currency"] == "N/A"].index, inplace=True)
data = data[data["Alternate Currency"].str.len() < 4]
data = data[data["Alternate Currency"].str.len() > 2]
alt_cur = data["Alternate Currency"].unique()

In [233]:
# Now we can perform the exchange between alternate currencies and USD
key = ""
url = "https://v6.exchangerate-api.com/v6/" + key + "/pair/"#+ "/latest/USD"
print(alt_cur)
ex_rat = {}
for ac in alt_cur:
    #break
    s = url + ac + "/USD"
    r = requests.get(s)#.json()["conversion_rate"]
    print(r)
    rjson = r.json()
    print(rjson)
    c = rjson["conversion_rate"]
    print(c)
    ex_rat[ac] = c

In [241]:
for i in data.index: 
    data["Total"][i] = data["Total"][i] * ex_rat[data["Alternate Currency"][i]]
data["Total"]
data.loc[data["Alternate Currency"] == "USD"]

/tmp/ipykernel_55/4187649414.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Total"][i] = data["Total"][i] * ex_rat[data["Alternate Currency"][i]]


,Age,Industry,Job Title,Currency,Alternate Currency,Country,US State,US City,Work experience,Work experience in field,Education,Gender,Race,Total
2473,25-34,Nonprofits,Special Events Manager,Other,USD,USA,Maine,Portland,8 - 10 years,8 - 10 years,College degree,Woman,White,48568.0
7402,45-54,Education (Higher Education),Executive Assistant,Other,USD,U.S.A.,Washington,Seattle,31 - 40 years,21 - 30 years,College degree,Woman,White,76500.0
9410,35-44,Retail,Sales Associate,Other,USD,USA,Massachusetts,Franklin,11 - 20 years,5-7 years,"Professional degree (MD, JD, etc.)",Woman,White,29600.0
10150,45-54,Property or Construction,Staff Accountant,Other,USD,USA,Illinois,Rockford,31 - 40 years,21 - 30 years,Some college,Woman,White,51500.0
11434,25-34,Law,L&D Specialist,Other,USD,USA,District of Columbia,"Washington, DC",5-7 years,5-7 years,"Professional degree (MD, JD, etc.)",Woman,Black or African American,91000.0
11788,45-54,Health care,RN,Other,USD,US,Oregon,Portland,11 - 20 years,5-7 years,College degree,Woman,White,120000.0
12481,45-54,"Accounting, Banking & Finance",Director of Human Resources,USD,USD,United States,Texas,Waco,11 - 20 years,11 - 20 years,College degree,Woman,White,97000.0
17588,25-34,Computing or Tech,Software Developer,USD,USD,USA,Washington,Seattle,8 - 10 years,5-7 years,High School,Man,Asian or Asian American,370000.0
19416,25-34,Computing or Tech,Data scientist,Other,USD,US,Illinois,Chicago,5-7 years,2 - 4 years,PhD,Woman,Asian or Asian American,135000.0
21036,25-34,Insurance,Talent Management Specialist II,Other,USD,USA,Texas,Houston,5-7 years,5-7 years,College degree,Woman,White,74000.0


In [242]:
# Finally, as every observation is now in the same currency, we can get rid of the columns Currency and Alternate Currency
data.drop(columns=["Currency", "Alternate Currency"], inplace=True)
data.head()

,Age,Industry,Job Title,Country,US State,US City,Work experience,Work experience in field,Education,Gender,Race,Total
434,25-34,Computing or Tech,Data analyst,India,NaN,Pune,2 - 4 years,2 - 4 years,Master's degree,Woman,Asian or Asian American,10699.650
603,45-54,Health care,Exec admin,Argentina,NaN,City of Buenos Aires,11 - 20 years,11 - 20 years,College degree,Woman,"Hispanic, Latino, or Spanish origin",5002.217
1311,25-34,Government and Public Administration,Associate,Malaysia,NaN,Kuala Lumpur,5-7 years,2 - 4 years,College degree,Woman,Another option not listed here or prefer not t...,21591.360
1840,35-44,Intergovernmental organization,Consultant,Switzerland,NaN,Geneva,11 - 20 years,5-7 years,Master's degree,Woman,Black or African American,67464.000
1915,35-44,Education (Primary/Secondary),Instructional coach,Kuwait,NaN,Hawalli,11 - 20 years,1 year or less,Master's degree,Woman,White,44168.988
